#  Aspect-Based & Opinion Mining

Problem: identify aspects and their sentiment orientation (ratings) from a set of reviews.
In this case the aspects are {food, service and location} and the reviews come from the Yelp dataset

In [10]:
import graphlab as graphlab
from graphlab import SFrame
from graphlab import SArray
import string
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
reviews = graphlab.SFrame.read_csv('yelp.csv')

# Text Pre-Processing

# Punctuation Removal

punctuation is removed, because for how the reviews are made it would not make any positive contribution

In [16]:
#Remove Punctuation
def remove_punctuation(text):
    return text.translate(None, string.punctuation)

reviews['text'] = reviews['text'].apply(remove_punctuation)

# Tokenization

Tokenization is the process of dividing text into a set of meaningful pieces and these pieces are called tokens.
Depending on the task at hand, we can define our own conditions to divide the input text into meaningful tokens.
it will be very useful as we will need to evaluate the individual words of each review

In [17]:
#Apply Tokenization
tokenized_reviews = graphlab.text_analytics.tokenize(reviews['text'])


# Stemming

In [18]:
#from NTLK
import nltk
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")

In [19]:
def word_stemmer(actual_word):
    ritorno = ''
    #try - except is needed because some characters are not recognized
    try:
        ritorno = stemmer.stem(actual_word)
    except UnicodeDecodeError:
        return actual_word
    return ritorno
    
    

tokenized_reviews = map(lambda s: map(lambda x: word_stemmer(x) ,s),tokenized_reviews)

In [20]:
reviews['text-tokenized'] = tokenized_reviews

# Word Count

the simplest way is to calculate the frequency of each word contained in each review

In [21]:
#Bag-of-Words dictionary, word -> word_count
dict_reviews = graphlab.text_analytics.count_words(reviews['text-tokenized'])


# Stop Words

In [22]:
#Remove stop-words
dict_reviews = dict_reviews.dict_trim_by_keys(graphlab.text_analytics.stopwords(), exclude=True)

# TF-IDF

Not all words are the same weight within reviews => give more weight to those that are characteristic for the review

In [23]:
#tf-idf using all the words in the reviews
tf_idf = graphlab.text_analytics.tf_idf(dict_reviews)
reviews['tf_idf'] = tf_idf

In [24]:
#Show result
reviews[:2]
#dict_reviews

Columns:
	business_id	str
	date	str
	review_id	str
	stars	int
	text	str
	type	str
	user_id	str
	cool	int
	useful	int
	funny	int
	text-length	int
	text-tokenized	list
	tf_idf	dict

Rows: 2

Data:
+------------------------+------------+------------------------+-------+
|      business_id       |    date    |       review_id        | stars |
+------------------------+------------+------------------------+-------+
| 9yKzy9PApeiPPOUJEtnvkg | 2011-01-26 | fWKvX83p0-ka4JS3dc6E5A |   5   |
| ZRJwVLyzEJq1VAihDhYiow | 2011-07-27 | IjZ33sJrzXqU-0X6U8NwyA |   5   |
+------------------------+------------+------------------------+-------+
+-------------------------------+--------+------------------------+------+--------+
|              text             |  type  |        user_id         | cool | useful |
+-------------------------------+--------+------------------------+------+--------+
| My wife took me here on my... | review | rLtl8ZkDX5vH5nAx9C3q5Q |  2   |   5    |
| I have no idea why some pe... | review | 0a2KyEL0d3Yb1V6aivbIuQ |  0   |   0    |
+-------------------------------+--------+------------------------+------+--------+
+-------+-------------+-------------------------------+
| funny | text-length |         text-tokenized        |
+-------+-------------+-------------------------------+
|   0   |    99193    | [my, wife, took, me, here,... |
|   0   |    66881    | [i, have, no, idea, whi, s... |
+-------+-------------+-------------------------------+
+-------------------------------+
|             tf_idf            |
+-------------------------------+
| {'ive': 3.768246152094924,... |
| {'pleas': 3.51965815370503... |
+-------------------------------+
[2 rows x 13 columns]

# Filtering of Relevant Reviews

All the reviews with 3 stars are eliminated as they tend to have a neutral sentiment

In [25]:
print("number of reviews before filtering: %d" % (len(reviews)))
reviews = reviews[reviews['stars'] != 3]
print("number of reviews after filtering: %d" % (len(reviews)))

number of reviews before filtering: 9963
number of reviews after filtering: 8505


# Natural Language Toolkit

NLTK is a platform for building python programs to work with human language data, it provides a suite of text processing libraries for classification, tokenization, stemming, tagging, parsing, and semantic reasoning, and more

In [26]:
from nltk.corpus import wordnet as wn
from nltk.tag import pos_tag_sents

# Categorize and Tagging Words

in the NLP is very important the difference between nouns, verbs, adjectives, and adverbs, tagging words. so that we use NLTK to go to classify each word in one of the word classes

In [27]:
reviews['food'] = pos_tag_sents(reviews['text-tokenized'])

# Terms Relevant to Each Aspect

In [28]:
important_for_food =" menu drink breakfast plate egg spaghetti calzone pizza sauce lunch dinner cook food grub meal eat fresh veggies crisp chow victuals spread Kitchen comestibles subsistence portion provisions cheer bread"
important_for_service = " service work prices ordered favor waiting use later tune-up ceremony amenity armed forces serve servicing help serving table service overhaul religious service military service "
important_for_place = " location dog Park country show home  situation seat job status weather responsibility space shoes position spot property piazza lieu station "

In [138]:
#per verificare l'efficacia delle frasi importanti selezionate
#map(lambda s: filter(lambda c: string.find(important_for_service, c) != -1, s), reviews['text-tokenized'])


# Relevant Elements

All elements referable to an adjective or an aspect are filtered

In [29]:
reviews_with_relevant_elements = list()
def adjective_and_aspect_filter(u):
    #first think to do is remove the stop-words because it isn't removed before (in tokenized element)
    if(u[0].lower() in graphlab.text_analytics.stopwords()):
        return ''
    if(string.find(u[1], 'JJ') != -1 or (string.find(important_for_food, u[0]) != -1) or (string.find(important_for_service, u[0]) != -1) or (string.find(important_for_place, u[0]))):
        return u
    else: 
        return ''

reviews_with_relevant_elements.append(map(lambda s: map(lambda u: adjective_and_aspect_filter(u) ,s), reviews['food']))

In [30]:
#normalizzazione di reviews_with_relevant_elements
reviews_with_relevant_elements = reviews_with_relevant_elements[0]

In [31]:
#filtriamo tutti quelli != ''
reviews_with_relevant_elements = map(lambda s: filter(lambda x: x != '', s), reviews_with_relevant_elements)

# Adjectives and Keywords

In [32]:
#filtraggio dei valori con lunghezza < 3 poiche classificati erroneamente come appartenenti ad una delle 3 categorie di interesse
#questo è dovuto al fatto che noi riusciamo a classificare in una categoria anche parole scritte male
adjectives_and_keywords = map(lambda s: filter(lambda x: len(x[0]) > 2, s), reviews_with_relevant_elements)

In [33]:
#smistamento degli aggettivi per le rispettive categorie
temporary_adjectives_for_the_cooking = ''
temporary_adjectives_for_service = ''
temporary_adjectives_for_location = ''

adjectives_cooking = list()
adjectives_service = list()
adjectives_location = list()

temporary_actual_adjectives = ''


def filtra_per_cucina(actual_word):
    global temporary_adjectives_for_the_cooking, temporary_actual_adjectives, temporary_adjectives_for_location, temporary_adjectives_for_service
    if(string.find(actual_word[1],'JJ') == -1):
        if(temporary_actual_adjectives != ''):
            if(string.find(important_for_place, actual_word[0]) != -1):
                temporary_adjectives_for_location = temporary_adjectives_for_location + temporary_actual_adjectives
                temporary_actual_adjectives = ''
            else:
                if(string.find(important_for_service, actual_word[0]) != -1):
                    temporary_adjectives_for_service = temporary_adjectives_for_service + temporary_actual_adjectives
                    temporary_actual_adjectives = ''
                else:
                    if(string.find(important_for_food, actual_word[0]) != -1):
                        temporary_adjectives_for_the_cooking = temporary_adjectives_for_the_cooking + temporary_actual_adjectives
                        temporary_actual_adjectives = ''
    else:
        temporary_actual_adjectives = temporary_actual_adjectives + " " + actual_word[0]
        
def init(actual_review):
    global temporary_adjectives_for_the_cooking, temporary_adjectives_for_location, temporary_adjectives_for_service, temporary_actual_adjectives
    temporary_actual_adjectives = ''
    map(lambda x: filtra_per_cucina(x), actual_review)
    #nel caso in cui vi siano rimasti aggettivi inerenti ad una review non assegnati a nessuna categoria essi
    #verranno associati alla cucina poichè molto probabilmente si riferisce alla cucina
  
    if(temporary_actual_adjectives != ''):
        temporary_adjectives_for_the_cooking = temporary_adjectives_for_the_cooking + temporary_actual_adjectives
    
    adjectives_cooking.append(temporary_adjectives_for_the_cooking)
    adjectives_service.append(temporary_adjectives_for_service)
    adjectives_location.append(temporary_adjectives_for_location)
    
    temporary_adjectives_for_the_cooking = ''
    temporary_adjectives_for_service = ''
    temporary_adjectives_for_location = ''

aggettivi_per = map(lambda s: init(s), adjectives_and_keywords)

# Split Into Three Indipendent SFrame 

In [34]:
#da list to list ad SArray
reviews['food'] = SArray(adjectives_cooking)
reviews['service'] = SArray(adjectives_service)
reviews['location'] = SArray(adjectives_location)

In [35]:
#preparazione SFrame per ogni categoria poichè così all'atto del training si può andare a valutare per ogni 
#categoria le reviews di interesse poichè può essere che in una categoria vi siano valori nulli

#SFrame per cucina
reviews_food = graphlab.SFrame({'food': adjectives_cooking})

#SFrame per servizio
reviews_service = graphlab.SFrame({'service': adjectives_service})

#SFrame per location
reviews_location = graphlab.SFrame({'location': adjectives_location})

In [36]:
#elimino tutti quelli che hanno campo vuoto '' così che non mi vadano ad addestrare male il classificatore
reviews_service = reviews_service[reviews_service['service'] != '']
reviews_food = reviews_food[reviews_food['food'] != '']
reviews_location = reviews_location[reviews_location['location'] != '']


In [37]:
#tokenizzazione
tokenized_food = graphlab.text_analytics.tokenize(reviews_food['food'])
tokenized_service = graphlab.text_analytics.tokenize(reviews_service['service'])
tokenized_location = graphlab.text_analytics.tokenize(reviews_location['location'])

reviews_food['food_tokenized'] = tokenized_food
reviews_service['service_tokenized'] = tokenized_service
reviews_location['location_tokenized'] = tokenized_location

In [38]:
#Bag-of-Words dictionary, word -> word_count
dict_reviews_food = graphlab.text_analytics.count_words(reviews_food['food_tokenized'])
dict_reviews_service = graphlab.text_analytics.count_words(reviews_service['service_tokenized'])
dict_reviews_location = graphlab.text_analytics.count_words(reviews_location['location_tokenized'])


In [39]:
#tf-idf using all the words in the reviews
tf_idf_food = graphlab.text_analytics.tf_idf(dict_reviews_food)
tf_idf_service = graphlab.text_analytics.tf_idf(dict_reviews_service)
tf_idf_location = graphlab.text_analytics.tf_idf(dict_reviews_location)

In [40]:
reviews_food['tf_idf_food'] = tf_idf_food
reviews_service['tf_idf_service'] = tf_idf_service
reviews_location['tf_idf_location'] = tf_idf_location

# Sentiment Analisys for Every Aspect of Interest

Important for calculating the sentiment columns

In [42]:
def predictions_mean(data, topic):
    m = None
    m = graphlab.sentiment_analysis.create(data, features=topic)
    predictions = m.predict(data)
    return predictions 
    

sentiment_food = predictions_mean(reviews_food,'food')
sentiment_service = predictions_mean(reviews_service,'service')
sentiment_location = predictions_mean(reviews_location,'location')

In [43]:
reviews_food['sentiment'] = sentiment_food.apply(lambda rating: +1 if rating > 0.5 else -1 )                                      
reviews_service['sentiment'] = sentiment_service.apply(lambda rating: +1 if rating > 0.5 else -1)
reviews_location['sentiment'] = sentiment_location.apply(lambda rating: +1 if rating > 0.5 else -1)

In [44]:
#sentiment for the reviews without the analysis of the aspects
#Introduce the sentiment -> +1 (good) for 4+ stars, -1 (bad) for 3- stars
reviews['sentiment'] = reviews['stars'].apply(lambda rating: +1 if rating > 3 else -1)

# Classificazioni

In [45]:
train_data_cucina, test_data_cucina = reviews_food.random_split(.8, seed = 0)
train_data_servizio, test_data_servizio = reviews_service.random_split(.8, seed = 0)
train_data_location, test_data_location = reviews_location.random_split(.8, seed = 0)
train_data_sentiment, test_data_sentiment = reviews.random_split(.8, seed = 0)

In [46]:
#svm_classifier
#cucina
features = ['tf_idf_food']
model_cucina = graphlab.svm_classifier.create(train_data_cucina, target = 'sentiment', 
                                       features = features,penalty=1.0, feature_rescaling = True,lbfgs_memory_level=7, max_iterations = 30)

#servizio
features = ['tf_idf_service']
model_servizio = graphlab.svm_classifier.create(train_data_servizio, target = 'sentiment', 
                                       features = features,penalty=1.0, feature_rescaling = True,lbfgs_memory_level=7, max_iterations = 30)

#location
features = ['tf_idf_location']
model_location = graphlab.svm_classifier.create(train_data_location, target = 'sentiment', 
                                       features = features,penalty=1.0, feature_rescaling = True,lbfgs_memory_level=7, max_iterations = 15)

#sentiment senza divisione del testo
#Model creation

features = ['tf_idf']
model_sentiment = graphlab.svm_classifier.create(train_data_sentiment, target = 'sentiment', 
                                       features = features,penalty=1.0, feature_rescaling = True,lbfgs_memory_level=7, max_iterations = 30)


PROGRESS: Creating a validation set from 5 percent of training data. This may take a while.
          You can set ``validation_set=None`` to disable validation tracking.



WARNING: The number of feature dimensions in this problem is very large in comparison with the number of examples. Unless an appropriate regularization value is set, this model may not provide accurate predictions for a validation/test set.

SVM:

--------------------------------------------------------

Number of examples          : 5860

...

| 6         | 13       | 1.000000  | 0.247101     | 0.992343          | 0.891026            |

| 11        | 18       | 1.000000  | 0.385165     | 0.998009          | 0.894231            |

+-----------+----------+-----------+--------------+-------------------+---------------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

In [47]:
def calculate_classification_test(test_data, model):
    predictions_margin = model.predict(test_data, output_type = 'margin')
    predictions_sign = model.predict(test_data)
    test_data['prediction_margin'] = predictions_margin
    test_data['prediction_sign'] = predictions_sign

calculate_classification_test(test_data_cucina, model_cucina)
calculate_classification_test(test_data_servizio, model_servizio)
calculate_classification_test(test_data_location, model_location)
calculate_classification_test(test_data_sentiment, model_sentiment)

# Accuracy

In [48]:
#accuratezza
def accuracy(test_data):
    num_correct = sum(test_data['sentiment'] == test_data['prediction_sign'])
    nc = float(num_correct)
    accuracy = nc / len(test_data)
    return float(accuracy)

accuracy = SFrame({'Food': [accuracy(test_data_cucina)], 'Service': [accuracy(test_data_servizio)], 'Location': [accuracy(test_data_location)], 'Sentiment': [accuracy(test_data_sentiment)]})

In [49]:
accuracy

Food,Location,Sentiment,Service
0.918340026774,0.9234375,0.871316897174,0.92972972973
